# WAVEWATCH III

[WAVEWATCH III](https://github.com/noaa-emc/ww3) is a community wave modeling framework that includes the latest scientific advancements in the field of wind-wave modeling and dynamics. The core of the framework consists of the WAVEWATCH III third-generation wave model (WAVE-height, WATer depth and Current Hindcasting), developed at NOAA/NCEP.


In fluid dynamics, wind wave modeling describes the effort to depict the sea state and predict the evolution of the energy of wind waves using numerical techniques. These simulations consider atmospheric wind forcing, nonlinear wave interactions, and frictional dissipation, and they output statistics describing wave heights, periods, and propagation directions for regional seas or global oceans. Such wave hindcasts and wave forecasts are extremely important for commercial interests on the high seas. 

For example, the shipping industry requires guidance for operational planning and tactical seakeeping purposes.

The model can incorporate the effects of currents on waves and is now extended for near shore applications.

In this notebook, we will use the [wavespectra library](https://wavespectra.readthedocs.io/en/latest/index.html), an open source project for processing ocean wave spectral data, to look at some outputs from the WAVEWATCH III model.

In [ ]:
import matplotlib.pyplot as plt
from wavespectra import read_ww3


%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['mathtext.fontset'] = 'cm'

### Loading WAVEWATCH III dataset

In [ ]:
dset = read_ww3("../../dataset/ww3file.nc")
dset.spec

### Spectral wave parameters

Wave energy density array named as `efth`, defined in:

- 2D spectra $E(\sigma,\theta)$ $m^{2}{degree^{-1}}{s}$.
- 1D spectra $E(\sigma)$ $m^{2}{s}$.

Several methods are available to calculate integrated wave parameters, for example to calculate the significant wave height one can use:

In [ ]:
hs = dset.spec.hs()

hs.plot.line(x="time")
plt.draw()

Using the `stats` methods we can evaluate the following:
    
+ hs Spectral significant wave height Hm0.
+ hmax Maximum wave height Hmax.
+ tp Peak wave period Tp.
+ tm01 Mean absolute wave period Tm01.
+ tm02 Mean absolute wave period Tm02.
+ dpm Peak wave direction Dpm.
+ dp Peak wave direction Dp.
+ dm Mean wave direction from the 1st spectral moment Dm.
+ dspr Directional wave spreading Dspr.
+ swe Spectral width parameter by Cartwright and Longuet-Higgins (1956).
+ sw Spectral width parameter by Longuet-Higgins (1975).

In [ ]:
stats = dset.spec.stats(
    ["hs", "hmax", "tp", "tm01", "tm02", "dpm", "dm", "dspr", "swe"]
)
stats

Let's plot the values from the `stats` method for the 2 sites available in our dataset:

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(12, 6))

stats.hs.plot.line(ax=ax1, x="time")


stats.hmax.plot.line(ax=ax2, x="time")
stats.dpm.plot.line(ax=ax3, x="time")
stats.dspr.plot.line(ax=ax4, x="time")
stats.tp.plot.line(ax=ax5, x="time")
stats.tm01.plot.line(ax=ax6, x="time")

ax1.set_ylabel('Hs(m)')
ax1.set_xticks([])
ax1.set_xlabel('')
ax2.set_ylabel('Hmax(m)')
ax2.set_xticks([])
ax2.set_xlabel('')
ax3.set_ylabel('Dpm(deg)')
ax3.set_xticks([])
ax3.set_xlabel('')
ax4.set_ylabel('Dspr(deg)')
ax4.set_xticks([])
ax4.set_xlabel('')
ax5.set_ylabel('Tp(s)')
ax6.set_ylabel('Tm01(s)')

plt.show()

## Wave partitioning

A wave component is defined as a region of enhanced energy in the directional wave spectrum S( f, θ) attributed to a windsea or swell system propagating from a specific wind-generation event on the ocean surface. 

Wave components are identified and extracted using spectral partitioning methods. Wave partitioning allows the identification and grouping of component wave systems from spatially and temporally distributed observations of directional wave spectra. 

The partition of wave spectra uses Hanson’s algorithm which depends on:

+ `wsp`: wind speed (m/s).
+ `wdir`: Wind direction (degree).
+ `dpt`: Water depth (m).
+ `swells`: Number of swell partitions to compute.


In [ ]:
dspart = dset.spec.partition(dset.wspd, dset.wdir, dset.dpt)
pstats = dspart.spec.stats(["hs", "dpm"])
pstats

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))
hs.isel(site=0).plot(ax=ax1, label='Full spectrum', marker='o')
pstats.hs.isel(part=0, site=0).plot(ax=ax1, label='Partition 0 (sea)', marker='o')
pstats.hs.isel(part=1, site=0).plot(ax=ax1, label='Partition 1 (swell 1)', marker='o')
pstats.hs.isel(part=2, site=0).plot(ax=ax1, label='Partition 2 (swell 2)', marker='o')
pstats.hs.isel(part=3, site=0).plot(ax=ax1, label='Partition 3 (swell 3)', marker='o')
ax1.set_ylabel('Hs (m)')
ax1.legend(bbox_to_anchor=(1.05, 1.05), frameon=False)
ax1.set_xticks([])
ax1.set_xlabel('')

dset.spec.dpm().isel(site=0).plot(ax=ax2, label='Full spectrum', marker='o')
pstats.dpm.isel(part=0, site=0).plot(ax=ax2, label='Partition 0 (sea)', marker='o')
pstats.dpm.isel(part=1, site=0).plot(ax=ax2, label='Partition 1 (swell 1)', marker='o')
pstats.dpm.isel(part=2, site=0).plot(ax=ax2, label='Partition 2 (swell 2)', marker='o')
pstats.dpm.isel(part=3, site=0).plot(ax=ax2, label='Partition 3 (swell 3)', marker='o')
ax2.set_ylabel('Dpm (deg)')
ax2.set_title('')

plt.show()

## Frequency-direction spectral plots in polar coordinates

One other option consists in splitting the spectra over freq .

+ `fmin`: lowest frequency to split spectra
+ `fmax`: highest frequency to split spectra

In [ ]:
sea = dset.spec.split(fmin=0.125)
swell = dset.spec.split(fmax=2)

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
dset.spec.hs().isel(site=0).plot(label='Full spectrum', marker='o')
sea.spec.hs().isel(site=0).plot(label='Sea', marker='o')
swell.spec.hs().isel(site=0).plot(label='Swell', marker='o')
ax.set_ylabel('Hs (m)')
ax.legend(bbox_to_anchor=(1.25, 1.05), frameon=False)
plt.draw()


fig, ax = plt.subplots(1, 1, figsize=(8, 4))
dset.spec.tp().isel(site=0).plot(label='Full spectrum', marker='o')
sea.spec.tp().isel(site=0).plot(label='Sea', marker='o')
swell.spec.tp().isel(site=0).plot(label='Swell', marker='o')
ax.set_ylabel('Tp (s)')
ax.legend(bbox_to_anchor=(1.25, 1.05), frameon=False)
plt.draw()

Plotting in polar coordinates

In [ ]:
ds = dset.isel(site=0, time=[0, -1]).spec.split(fmin=0.125, fmax=2)

ds.spec.plot(
     kind="contourf",
     col="time",
     as_period=False,
     normalised=True,
     logradius=True,
     add_colorbar=False,
     figsize=(8, 5));